#Chapter 15

word similarity / Cosine Similarity

2 words 

w1 [.....], w2 [.....]

make these into a coordinate 

compute the similarity between the two vectors

In [1]:
import dill
import numpy as np

def saveModel(model,fileNameDotpkl):
    with open(fileNameDotpkl,'wb') as f:
        dill.dump(model,f)
def loadModel(fileNameDotpkl):
    with open(fileNameDotpkl,'rb') as f:
        return dill.load(f)

E,W2I,I2W,vocab = loadModel('../data/w2v_100d.pkl')

E.shape

#E[W2I['azure'],:]

(108, 100)

In [2]:
def getNorms(E):
    if E.ndim == 1:
        E = E[np.newaxis,:]
    nrms = np.sum(E**2,axis=1)**0.5
    return nrms

def normalize(E):
    if E.ndim == 1:
        E = E[np.newaxis,:]
    nrms = getNorms(E)
    return E/nrms[:,np.newaxis]

def cosineSimilarity(E,v):
    E = normalize(E)
    v = normalize(v)
    scores = E.dot(v.T)
    return scores

def getMostSimilarWords(E,word,W2I,topn=10):
    if type(word) is str:
        v = E[W2I[word],:]
    else:
        v = word
    scores = cosineSimilarity(E,v)
    scores = scores.squeeze()
    sortedScores = np.sort(scores)[::-1]
    idx = np.argsort(scores)[::-1]
    topNScores = sortedScores[:topn]
    topNWordsIdx = idx[:topn]
    return topNScores,topNWordsIdx
    
def analogy(E,W2I,man,woman,king):
    v = E[W2I[king],:]-E[W2I[man],:]+E[W2I[woman],:]
    scores,idx = getMostSimilarWords(E,v,W2I)
    return scores,idx

scores,idx = analogy(E,W2I,'project','application','pipelines')
for i in range(len(idx)):
    print(scores[i],I2W[idx[i]])

0.6777922792556244 application
0.5682181659339786 pipelines
0.5424042115595893 container
0.5106526802270114 setting
0.42110082384235037 pipeline
0.41397067200104903 up
0.37999758369347403 for
0.3799119565420121 hosted
0.3707696638977539 based
0.35721198279617694 agents


## Word Visualization

pip install plotly
pip install nbformat

had to restart after installing nbformat.

In [3]:
from sklearn.decomposition import PCA

def reduceDim(X,new_dim=3):
    pca = PCA(n_components=new_dim)
    pca.fit(X)
    X_new = pca.transform(X)
    return X_new

X = np.random.rand(50,100)
print(X.shape)


X_new = reduceDim(X)
print(X_new.shape)

def sampleWords(E,W2I,wordsToSample):
    indx = [W2I[word] for word in wordsToSample]
    E_sampled = E[indx,:]
    return E_sampled

wordsToSample = ['azure', 'devops', 'with', 'and', 'your', 'all', 'of', 'this', 'be', 'in' ,'a' ,'or',
 'by', 'the', 'to', 'is' ,'s' ,'will' ,'for' ,'have' ,'use' ,'development',
 'project', '1', '3', 'com' ,'our' ,'as', 'you' ,'from', '4', 'that' ,'can' ,'at',
 'are', 'on', 'also', 'up', 'microsoft', 'an' ,'it' ,'now', 'work', 'using',
 'application', '6', 'new', 'if' ,'we' ,'section' ,'2', 'create', '5' ,'ci' ,'cd',
 'control', 'code', 'boards', 'pipelines' ,'test', 'artifacts', 'creating',
 'organization', 'items', 'source', 'github', 'repository' ,'pull' ,'request',
 'branch' ,'visual', 'studio' ,'release', 'build' ,'agents', 'hosted' ,'when',
 'agent' ,'pipeline', 'stage', 'container' ,'testing' ,'setting', 'set',
 'package', 'deployment', 'based', 'service' ,'chapter', 'how' ,'add' ,'need',
 'following', 'https', 'next' ,'used', 'here', 'run', 'name', 'see', 'select',
 'then', 'different', 'figure', 'll', 'created', 'task', 'click']

E_sampled = sampleWords(E,W2I,wordsToSample)

print(E_sampled.shape)

E_sampled_new = reduceDim(E_sampled)

print(E_sampled_new.shape)


import plotly.express as px
import pandas as pd

def plot3D(X,Y,Z,labels):
    df = pd.DataFrame({'X':X,'Y':Y,'Z':Z,'C':labels})
    fig = px.scatter_3d(df,x='X',y='Y',z='Z',color='C')
    fig.show()

plot3D(E_sampled_new[:,0],E_sampled_new[:,1],E_sampled_new[:,2],wordsToSample)

(50, 100)
(50, 3)
(108, 100)
(108, 3)
